<a href="https://colab.research.google.com/github/women-in-ai-ireland/September-2022-CeADAR-Group-002/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dicCounties={'Carlow':['Carlow'],'Cavan':['Cavan'],'Clare':['Clare'],'Cork':['Cork'],'Donegal':['Donegal'],'Dublin':['Dublin','Fingal','Dún Laoghaire'],
             'Galway':['Galway'],'Kerry':['Kerry'],'Kildare':['Kildare'],'Kilkenny':['Kilkenny'],'Laois':['Laois'],'Leitrim':['Leitrim'],
             'Limerick':['Limerick'],'Longford':['Longford'],'Louth':['Louth'],'Mayo':['Mayo'],'Meath':['Meath'],'Monaghan':['Monaghan'],
             'Offaly':['Offaly'],'Roscommon':['Roscommon'],'Sligo':['Sligo'],'Tipperary':['Tipperary'],'Waterford':['Waterford'],
             'Westmeath':['Westmeath'],'Wexford':['Wexford'],'Wicklow':['Wicklow']};

# lstQuarters=['2011Q1','2011Q2','2011Q3','2011Q4','2012Q1','2012Q2','2012Q3','2012Q4','2013Q1','2013Q2','2013Q3','2013Q4','2014Q1','2014Q2','2014Q3','2014Q4',
#              '2015Q1','2015Q2','2015Q3','2015Q4','2016Q1','2016Q2','2016Q3','2016Q4','2017Q1','2017Q2','2017Q3','2017Q4','2018Q1','2018Q2','2018Q3','2018Q4',
#              '2019Q1','2019Q2','2019Q3','2019Q4','2020Q1','2020Q2','2020Q3','2020Q4','2021Q1','2021Q2','2021Q3','2021Q4','2022Q1','2022Q2'];

Quarters={'Q1':['January','February','March'],
          'Q2':['April','May','June'],
          'Q3':['July','August','September'],
          'Q4':['October','November','December']
          }

In [3]:
dfNDQ06=pd.read_csv("https://raw.githubusercontent.com/women-in-ai-ireland/September-2022-CeADAR-Group-002/main/dataset/NDQ06.20221013.csv")
dfNDQ06=dfNDQ06[dfNDQ06["Type of House"]=="All house types"]

dfNDQ06['County']=np.nan

for county in dicCounties:
  filter=dfNDQ06['Local Authority'].str.contains('|'.join(dicCounties[county]),case=False,regex=True,na=False)
  dfNDQ06.loc[filter, 'County'] = county

dfNDQ06.drop(columns=['STATISTIC','Type of House','UNIT','Local Authority'],inplace=True)
dfNDQ06.rename(columns={'VALUE':'New_House_n'},inplace=True)
dfNDQ06=dfNDQ06.groupby(['County','Quarter']).sum()

dfGrand=dfNDQ06.copy()

In [4]:
dfHPM02=pd.read_csv("https://raw.githubusercontent.com/women-in-ai-ireland/September-2022-CeADAR-Group-002/main/dataset/HPM02.20221018.csv")
dfHPM02=dfHPM02[dfHPM02["Dwelling Status"]=="All Dwelling Statuses"]
dfHPM02=dfHPM02[dfHPM02["Stamp Duty Event"]=="Executions"]
dfHPM02.drop(columns=['Dwelling Status','Type of Buyer','Type of Sale','Stamp Duty Event'],inplace=True)
dfHPM02

def setQuarter(month):
  split=month.split()
  quarter = [i for i in Quarters if split[1] in Quarters[i]]
  return split[0]+quarter[0]


dfHPM02["Quarter"]=dfHPM02['Month'].apply(setQuarter)

dfHPM02tmp=dfHPM02[dfHPM02['Statistic']=='Volume of Sales']
dfGrand['Sales_Volume_n']=dfHPM02tmp.groupby(['County','Quarter']).sum().astype(int)

dfHPM02tmp=dfHPM02[dfHPM02['Statistic']=='Value of Sales']
dfGrand['Sales_Value_MEur']=dfHPM02tmp.groupby(['County','Quarter']).sum()

dfHPM02tmp=dfHPM02[dfHPM02['Statistic']=='Median Price']
dfGrand['Median_Price_Eur']=dfHPM02tmp.groupby(['County','Quarter']).mean().astype(int)

dfHPM02tmp=dfHPM02[dfHPM02['Statistic']=='Mean Sale Price']
dfGrand['Mean_Price_Eur']=dfHPM02tmp.groupby(['County','Quarter']).mean().astype(int)

In [5]:
dfE1071=pd.read_csv("https://raw.githubusercontent.com/women-in-ai-ireland/September-2022-CeADAR-Group-002/main/dataset/E1071.20221013.csv")
dfFP012=pd.read_csv("https://raw.githubusercontent.com/women-in-ai-ireland/September-2022-CeADAR-Group-002/main/dataset/FP012.20221013.csv")

dfE1071['Quarter']=dfE1071['CensusYear'].apply(lambda x:str(x)+'Q2')
dfE1071['County']=np.nan
for county in dicCounties:
  filter=dfE1071['County and City'].str.contains('|'.join(dicCounties[county]),case=False,regex=True,na=False)
  dfE1071.loc[filter, 'County'] = county

dfE1071=dfE1071.set_index(['County','Quarter'])

dfFP012['Quarter']=dfFP012['CensusYear'].apply(lambda x:str(x)+'Q2')
dfFP012['County']=np.nan
for county in dicCounties:
  filter=dfFP012['Admin Counties'].str.contains('|'.join(dicCounties[county]),case=False,regex=True,na=False)
  dfFP012.loc[filter, 'County'] = county

dfFP012=dfFP012.set_index(['County','Quarter'])

In [6]:
dfE1071tmp=dfE1071[dfE1071['Statistic']=='Total housing stock']
i=dfE1071tmp.groupby(['County','Quarter']).sum() 
dfGrand['Stock_House_n']=i['VALUE']

dfE1071tmp=dfE1071[dfE1071['Statistic']=='Vacant']
i=dfE1071tmp.groupby(['County','Quarter']).sum() 
dfGrand['Vacant_House_n']=i['VALUE']

dfFP012tmp=dfFP012[dfFP012['Statistic']=='Housing Stock']
i=dfFP012tmp.groupby(['County','Quarter']).sum()
dfGrand.loc[(slice(None),'2022Q2'),'Stock_House_n']=i['VALUE']

dfFP012tmp=dfFP012[dfFP012['Statistic']=='Vacant Dwellings']
i=dfFP012tmp.groupby(['County','Quarter']).sum()
dfFP012tmp=dfFP012[dfFP012['Statistic']=='Unoccupied Holiday Homes']
j=dfFP012tmp.groupby(['County','Quarter']).sum()
dfGrand.loc[(slice(None),'2022Q2'),'Vacant_House_n']=i['VALUE']+j['VALUE']


for county in dfGrand.index.get_level_values(0).unique():
  dfGrand.loc[county]['Stock_House_n']=dfGrand.loc[county,'Stock_House_n'].interpolate(method='linear',limit_direction='both')
  dfGrand.loc[county]['Vacant_House_n']=dfGrand.loc[county,'Vacant_House_n'].interpolate(method='linear',limit_direction='both')

dfGrand['Stock_House_n']=dfGrand['Stock_House_n'].astype(int)
dfGrand['Vacant_House_n']=dfGrand['Vacant_House_n'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
dfFP002=pd.read_csv("https://raw.githubusercontent.com/women-in-ai-ireland/September-2022-CeADAR-Group-002/main/dataset/FP002.20221025.csv")
dfFP002=dfFP002[dfFP002['Statistic']=='Population']
dfFP002['Quarter']=dfFP002['CensusYear'].apply(lambda x:str(x)+'Q2')
dfFP002.rename(columns={'County':'County_co'},inplace=True)
dfFP002['County']=np.nan
for county in dicCounties:
  filter=dfFP002['County_co'].str.contains('|'.join(dicCounties[county]),case=False,regex=True,na=False)
  dfFP002.loc[filter, 'County'] = county

dfFP002=dfFP002.set_index(['County','Quarter'])

i=dfFP002.groupby(['County','Quarter']).sum() 
dfGrand['Population']=i['VALUE']

for county in dfGrand.index.get_level_values(0).unique():
  dfGrand.loc[county]['Population']=dfGrand.loc[county,'Population'].interpolate(method='linear',limit_direction='both')

dfGrand['Population']=dfGrand['Population'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [8]:
dfGrand.to_csv('HousingData.csv')
dfGrand

New_House_n  Sales_Volume_n  Sales_Value_MEur  \
County  Quarter                                                  
Carlow  2011Q1            17              50               6.8   
        2011Q2            24              59               6.6   
        2011Q3            24              62               5.9   
        2011Q4            38              74               6.1   
        2012Q1            17              48               3.9   
...                      ...             ...               ...   
Wicklow 2021Q2           149             526             186.2   
        2021Q3           229             639             250.5   
        2021Q4           239             697             281.5   
        2022Q1           268             420             151.1   
        2022Q2           332             591             255.4   

                 Median_Price_Eur  Mean_Price_Eur  Stock_House_n  \
County  Quarter                                                    
Carlow  2011Q1             153205          136542          23165   
        2011Q2             110000          111140          23165   
        2011Q3              77833           96286          23170   
        2011Q4              71848           81263          23175   
        2012Q1              77666           80014          23181   
...                           ...             ...            ...   
Wicklow 2021Q2             323500          348426          59258   
        2021Q3             342500          392000          59472   
        2021Q4             375116          401860          59685   
        2022Q1             337367          361707          59899   
        2022Q2             403888          431266          60113   

                 Vacant_House_n  Population  
County  Quarter                              
Carlow  2011Q1             3202       54612  
        2011Q2             3202       54612  
        2011Q3             3150       54728  
        2011Q4             3098       54844  
        2012Q1             3046       54960  
...                         ...         ...  
Wicklow 2021Q2             4446      153308  
        2021Q3             4446      153852  
        2021Q4             4445      154396  
        2022Q1             4445      154940  
        2022Q2             4445      155485  

[1196 rows x 8 columns]